In [1]:
import numpy as np

In [2]:
text = "The dog chased the cat"

In [3]:
dataPipes1 = np.zeros(shape=[5, 6])
dataPipes1

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

# self attention layer

## introduction

#### IF X = token embeddings / hidden states ของคำทั้งหมด

In [4]:
TheWord = np.array([1, 0, 1, 0, 0, 0])
dogWord = np.array([0, 0, 1, 0, 0, 0])
chasedWord = np.array([0, 1, 0, 0, 0, 0])
theWord = np.array([0, 1, 1, 0, 0, 0])
catWord = np.array([1, 0, 0, 0, 0, 0])

In [5]:
X = np.array([TheWord, dogWord, chasedWord, theWord, catWord])
print(X)

[[1 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 1 0 0 0 0]
 [0 1 1 0 0 0]
 [1 0 0 0 0 0]]


In [6]:
dataPipes1 = X

#### K Q V ถูกสร้างจาก X ดังนี้
* $Q = X \cdot W_q$ - Q (query) : ฉันกำลังมองหาอะไร
* $K = X \cdot W_k$ - K (Key) : ฉันมีอะไรให้คนอื่นสนใจ
* $V = X \cdot W_v$ - V (value) : ข้อมูลจริงที่จะส่งต่อ

X คือคำ  
Q คือคำถาม  
K คือป้ายชื่อ  
V คือเนื้อหา

* $X ∈ R^{(n × d_{model})}$  
* $n$ = จำนวน tokens    
* $d_{model}$ = embedding size  
* $W_q ∈ R^{(d_{model} × d_k)}$  
* $W_k ∈ R^{(d_{model} × d_k)}$  
* $W_v ∈ R^{(d_{model} × d_v)}$  

In [7]:
n = 5 # 5 คำ ให้เป็น 5 โทเคน
d_model = 6 # example
num_heads = 2 # จำนวน attentions mattrix ที่ทำการคำนวณขนานกัน

In [8]:
#d_k = d_v = d_model / num_heads
d_k = int(d_model / num_heads)
d_v = int(d_model / num_heads)

In [9]:
W_Q1 = np.random.rand(d_model, d_k)
W_K1 = np.random.rand(d_model, d_k)
W_V1 = np.random.rand(d_model, d_v)

In [10]:
W_Q1

array([[0.07582243, 0.30979567, 0.7430575 ],
       [0.22170115, 0.43361163, 0.7879522 ],
       [0.86550923, 0.33813209, 0.89389491],
       [0.3916188 , 0.09192825, 0.28173238],
       [0.66579822, 0.41607189, 0.81859121],
       [0.87640074, 0.23404346, 0.77525916]])

In [11]:
W_Q2 = np.random.rand(d_model, d_k)
W_K2 = np.random.rand(d_model, d_k)
W_V2 = np.random.rand(d_model, d_v)

In [12]:
W_Q2

array([[0.93130415, 0.98743468, 0.58815925],
       [0.59675434, 0.26268293, 0.4522365 ],
       [0.41705605, 0.93156201, 0.8470964 ],
       [0.14309467, 0.19419663, 0.71733839],
       [0.90737717, 0.60025281, 0.8901637 ],
       [0.26206242, 0.89711944, 0.90616465]])

### attention #1

In [13]:
Q1 = np.dot(X, W_Q1)
K1 = np.dot(X, W_K1)
V1 = np.dot(X, W_V1)
Q1

array([[0.94133166, 0.64792776, 1.63695241],
       [0.86550923, 0.33813209, 0.89389491],
       [0.22170115, 0.43361163, 0.7879522 ],
       [1.08721038, 0.77174372, 1.68184711],
       [0.07582243, 0.30979567, 0.7430575 ]])

### attention #2

In [14]:
Q2 = np.dot(X, W_Q2)
K2 = np.dot(X, W_K2)
V2 = np.dot(X, W_V2)
Q2

array([[1.3483602 , 1.91899669, 1.43525565],
       [0.41705605, 0.93156201, 0.8470964 ],
       [0.59675434, 0.26268293, 0.4522365 ],
       [1.01381038, 1.19424494, 1.2993329 ],
       [0.93130415, 0.98743468, 0.58815925]])

### softmax
    

In [15]:
def softmax(x):
    x = x - np.max(x) # stability ป้องกันตัวเลข Overflow เป็นมาตรฐานใน deeplearning เพื่อป้องกันไม่ให้ค่าเฟ้อจากฟังก์ชัน exponential
    # print("current x is : ")
    # print(x)
    exp_x = np.exp(x)
    # print("current exp_x is : ")
    # print(exp_x)
    return exp_x / np.sum(exp_x)

## Attention Head 1

#### attention score of head 1

In [16]:
print(Q1.shape)
print(K1.shape)

(5, 3)
(5, 3)


$$score = \frac{Q \cdot K^T}{d_K} $$

In [17]:
scores1 = np.dot(Q1, K1.T) / np.sqrt(d_k)
scores1

array([[1.47283138, 0.54704292, 0.84279087, 1.38983379, 0.92578847],
       [0.90671043, 0.33171338, 0.46829882, 0.80001221, 0.57499705],
       [0.71496941, 0.25086223, 0.41398267, 0.6648449 , 0.46410717],
       [1.62167984, 0.58257561, 0.88228149, 1.46485711, 1.03910423],
       [0.56612095, 0.21532954, 0.37449205, 0.58982158, 0.35079141]])

$$scores ∈ R^{(n_{token} \times n_{token})}$$

#### apply softmax row by row 

In [18]:
attn1 = np.apply_along_axis(softmax, axis=1, arr=scores1)

In [19]:
attn1

array([[0.29173223, 0.11558982, 0.1553679 , 0.26849674, 0.16881331],
       [0.26148873, 0.14714141, 0.16867598, 0.23502525, 0.18766863],
       [0.24402003, 0.15341446, 0.18059619, 0.23209013, 0.1898792 ],
       [0.30821521, 0.10903774, 0.14714227, 0.26347949, 0.17212529],
       [0.22935763, 0.16149773, 0.18936068, 0.23485848, 0.18492548]])

In [20]:
output1 = np.dot(attn1, V1)
output1

array([[0.48636   , 0.85373799, 0.25661462],
       [0.46682243, 0.81996578, 0.24926628],
       [0.46085022, 0.80514242, 0.24267155],
       [0.48992633, 0.8645315 , 0.2648232 ],
       [0.45705727, 0.79446875, 0.23443277]])

##### อธิบายความหมายของค่าที่ได้

In [21]:

#โฟกัสที่คำว่า "chased"

word_index = 2 # chased
print("attention weight: ", attn1[word_index])

attention weight:  [0.24402003 0.15341446 0.18059619 0.23209013 0.1898792 ]


ตรงนี้มี่ความหมายว่าคำว่า "chased" มีความสนใจคำอื่นมากแค่ไหน (อาจหมายถึงคำว่า "chased" มีความสัมพันธ์กับคำอื่นแค่ไหน)

In [22]:
print("output vector: ", output1[word_index])

output vector:  [0.46085022 0.80514242 0.24267155]


เป็นเวคเตอร์ใหม่ของคำว่า "chased" ที่มีบริบทรวมอยู่เรียบร้อยแล้ว

## Attention head 2

In [23]:
scores2 = np.dot(Q2, K2.T)/np.sqrt(d_k)
print("scores from second attendtion head 2 ->")
print(scores2)

attn2 = np.apply_along_axis(softmax, axis=1, arr=scores2);
print("attention2 =>")
print(attn2)

output2 = np.dot(attn2, V2)
print("output 2 =>");
print(output2)

scores from second attendtion head 2 ->
[[3.29954425 1.69490639 1.5581178  3.25302419 1.60463785]
 [1.51227562 0.78171183 0.7130871  1.49479893 0.73056379]
 [0.99628244 0.56895798 0.36783681 0.9367948  0.42732446]
 [2.50855806 1.35066981 1.08092391 2.43159373 1.15788825]
 [1.78726863 0.91319456 0.8450307  1.75822526 0.87407407]]
attention2 =>
[[0.39771009 0.07992474 0.06970673 0.37963234 0.0730261 ]
 [0.29659261 0.14285008 0.13337584 0.29145418 0.13572729]
 [0.27070674 0.17656898 0.14440047 0.25507266 0.15325115]
 [0.36509486 0.11469414 0.08757741 0.33804968 0.09458391]
 [0.31450323 0.1312259  0.12257909 0.30550037 0.1261914 ]]
output 2 =>
[[0.75908418 1.17162671 1.37390062]
 [0.68368998 1.0260638  1.23035377]
 [0.65006466 0.99259854 1.18401833]
 [0.72214468 1.12628974 1.31847193]
 [0.69534792 1.05125232 1.25501192]]


## Now we got the 2 output from both attention head 1 and 2

#### let concat it together

In [24]:
output = np.concatenate([output1, output2], axis=1)
output

array([[0.48636   , 0.85373799, 0.25661462, 0.75908418, 1.17162671,
        1.37390062],
       [0.46682243, 0.81996578, 0.24926628, 0.68368998, 1.0260638 ,
        1.23035377],
       [0.46085022, 0.80514242, 0.24267155, 0.65006466, 0.99259854,
        1.18401833],
       [0.48992633, 0.8645315 , 0.2648232 , 0.72214468, 1.12628974,
        1.31847193],
       [0.45705727, 0.79446875, 0.23443277, 0.69534792, 1.05125232,
        1.25501192]])

In [25]:
output.shape

(5, 6)

In [26]:
X.shape

(5, 6)

In [27]:
dataPipes1 = output

now output from attention layer have the same dimension and size as X

## Output projection after concat output

**Output Projection** คือ **Linear transformation** สุดท้ายหลังจากรวม (concat) ผลลัพธ์ของทุก attention heads 
เพื่อผสมมุมมองทั้งหมด และทำให้ขนาดกลับไปเป็น $d_{model}$

สัญลักษณ์ใน Paper คือ $W_O$

ดำเนินการหลังจาก concat เป็น output จาก multi-head attention เรียบร้อยแล้วโดยการ

$$Output_{final} = Concat(output_1, output_2, ... , output_n) \cdot W_O$$

* ส่วนนี้เกิดขึ้นมาจากการที่แต่ละ Head ทำงานแยกขาดจากกัน
* ทำให้ไม่มีการผสมข้อมูลระหว่างแต่ละ attention head 
* ทำให้โมเดลเสียความสามารถในการเรียนรู้

**Output Projection** ทำหน้าที่
* ผสมข้อมูลจากทุก Head 
* เรียนรู้ว่าควรเน้นข้อมูลจาก Head ไหน
* จัด alignment กลับเข้าสู่ space หลัก ($d_{model}$)

In [28]:
#example W_O

W_O = np.random.rand(d_model, d_model)
W_O

array([[0.77583616, 0.01549729, 0.45127571, 0.79148822, 0.96174933,
        0.43994339],
       [0.59896472, 0.54823805, 0.36543669, 0.99840838, 0.87790643,
        0.52692263],
       [0.15674411, 0.99919369, 0.62073693, 0.81194203, 0.29711081,
        0.02112464],
       [0.59786167, 0.01051385, 0.49964086, 0.93859613, 0.06025412,
        0.58843722],
       [0.85812154, 0.2667245 , 0.19671005, 0.55021675, 0.9457751 ,
        0.65250247],
       [0.54814432, 0.06728198, 0.79271746, 0.25299618, 0.64459704,
        0.76327123]])

In [29]:
attention_output = np.dot(output, W_O)
attention_output

array([[3.14123873, 1.14491783, 2.38961505, 3.15039731, 3.33294705,
        2.92906715],
       [2.85603024, 1.06948142, 2.18379942, 2.908076  , 3.04758223,
        2.65361106],
       [2.76726536, 1.04227565, 2.11147926, 2.82149832, 2.96332121,
        2.56604448],
       [3.06038492, 1.14288226, 2.32895014, 3.09702155, 3.26745801,
        2.84287069],
       [2.87295805, 1.04903011, 2.19119395, 2.89388625, 3.05181908,
        2.67768302]])

**from paper :**  
Output projection lets the model recombine information from different attention heads into a unified representation.

## Add & Norm step after get final output from multi-head attention

ใน 1 Transformer block จะมีการ Add & Norm 2 ครั้ง

ครั้งแรกเกิดหลังจากการทำ Multi-head Attention
ครั้งที่สองเกิดหลังการทำ Feed-Forward Network

โดยการนำ Embedding Vector หรือ X มาบวกเข้ากับ Final output จาก multi-head attention แล้วทำการ Normalized เพื่อให้ค่าใน Vector ไม่โตจนเกินไป

### การ ADD

$$Y = X + F(X)$$
โดย 
* $X$ คือ input เดิม
* $F(X)$ คือ output จาก multi-head attention or FFN

กระบวนการนี้เกิดจากการที่
* เมื่อค่าที่ได้จาก transformer block ผ่านไปยังอีก block ที่อยู่ลึกมาก ๆ แล้ว gradient หายไประหว่างทาง (Vanishing gradient)
* ทำให้การ training ทำได้ยาก
* และข้อมูลเดิมหายระหว่างทาง

โมเดลควรเรียนรู้ว่า "ฉันควรเพิ่มอะไรจากของเดิม"

In [30]:
residual1 = X + dataPipes1
residual1

array([[1.48636   , 0.85373799, 1.25661462, 0.75908418, 1.17162671,
        1.37390062],
       [0.46682243, 0.81996578, 1.24926628, 0.68368998, 1.0260638 ,
        1.23035377],
       [0.46085022, 1.80514242, 0.24267155, 0.65006466, 0.99259854,
        1.18401833],
       [0.48992633, 1.8645315 , 1.2648232 , 0.72214468, 1.12628974,
        1.31847193],
       [1.45705727, 0.79446875, 0.23443277, 0.69534792, 1.05125232,
        1.25501192]])

In [31]:
X

array([[1, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0]])

In [32]:
residual1

array([[1.48636   , 0.85373799, 1.25661462, 0.75908418, 1.17162671,
        1.37390062],
       [0.46682243, 0.81996578, 1.24926628, 0.68368998, 1.0260638 ,
        1.23035377],
       [0.46085022, 1.80514242, 0.24267155, 0.65006466, 0.99259854,
        1.18401833],
       [0.48992633, 1.8645315 , 1.2648232 , 0.72214468, 1.12628974,
        1.31847193],
       [1.45705727, 0.79446875, 0.23443277, 0.69534792, 1.05125232,
        1.25501192]])

### Norm (Layer Normalization) 


ทำให้ค่าใน vector 
* $mean \approx 0$
* $variance \approx 1$

$$LayerNorm(x) = \frac{x - \mu}{\sigma} \cdot \gamma + \beta$$

ในกรณีที่ไม่มีการ Normalized 
* ทำให้ Vector โตขึ้นเรื่อย ๆ
* scale เพี้ยนเรื่อย ๆ 
* training ไม่นิ่ง

In [33]:
def layer_norm (x, eps=1e-6):
    mean = np.mean(x, axis=1, keepdims=True)
    print("mean = ", mean)
    std = np.std(x, axis=1, keepdims=True)
    print("std = ", std)
    return (x - mean) / (std + eps)

In [34]:
norm1 = layer_norm(residual1)

mean =  [[1.15022069]
 [0.91269367]
 [0.88922429]
 [1.13103123]
 [0.91459516]]
std =  [[0.26321411]
 [0.28486376]
 [0.51562711]
 [0.44145274]
 [0.39873644]]


In [35]:
norm1

array([[ 1.27705176, -1.12638937,  0.40420907, -1.48599566,  0.08132521,
         0.84979899],
       [-1.56520326, -0.32551549,  1.18151718, -0.8039032 ,  0.39797878,
         1.115126  ],
       [-0.83078105,  1.77631536, -1.2539129 , -0.46382194,  0.2004822 ,
         0.57171833],
       [-1.45225839,  1.66155636,  0.30307132, -0.9262274 , -0.01074062,
         0.42459873],
       [ 1.36044942, -0.30126693, -1.70579014, -0.54985366,  0.34272468,
         0.85373664]])

### eps value

eps = $\epsilon$ 
* เป็นค่าคงที่เล็ก ๆ ใช้ป้องกันการหารด้วย 0
* มักจะมีค่าเท่ากับ $1 \times 10^{-6}$ เนื่องจากเป็นค่าที่เล็กพอที่จะไม่รบกวนผลลัพธ์แต่ใหญ่พอที่จะป้องกัน numerical error

### $\beta$ and $\gamma$ หายไปไหน

$\beta$ และ $\gamma$
เป็นค่าที่ได้จากการ training 
ทำให้โมเดลสามารถเลือกได้ว่า 
* dimension ไหนควรเพิ่ม - ลด และ 
* dimension ไหนควรเลื่อนค่า

In [36]:
#for example beta and gamma after training method is 
beta1 = np.array([2, 0.5, 2, 1, 1, 1])
gamma1 = np.array([0.1, -0.2, 0.3, -0.12, 0.32, 0.2])

In [37]:
y = gamma1 * norm1 + beta1
y

array([[2.12770518, 0.72527787, 2.12126272, 1.17831948, 1.02602407,
        1.1699598 ],
       [1.84347967, 0.5651031 , 2.35445515, 1.09646838, 1.12735321,
        1.2230252 ],
       [1.91692189, 0.14473693, 1.62382613, 1.05565863, 1.0641543 ,
        1.11434367],
       [1.85477416, 0.16768873, 2.0909214 , 1.11114729, 0.996563  ,
        1.08491975],
       [2.13604494, 0.56025339, 1.48826296, 1.06598244, 1.1096719 ,
        1.17074733]])

In [38]:
dataPipes1 = norm1

# Feed Forward neural network

สิ่งที่ได้จากกระบวนการนี้
$$H \in R^{(n \times d_{model})}$$

* FFN ทำงานกับ H ทีละ token (row-wise)
* FFN จะไม่ดู token อื่นเลย (ไม่ขึ้นกับ token อื่น)

FFN definition from Attention is All you need: 
    $$FFN(x) = max(0, x \cdot W_1 + b_1)W_2 + b_2$$

(ของใหม่จะใช้ GELU แทน ReLU)

## 1st linear (expand)

expand dimension from 6 to 24 (4 times of $n_{model}$)

In [39]:
# this value came from training method
W_FFN_Expand_1 = np.random.rand(6, 24) 
W_FFN_Shrink_1 = np.random.rand(24, 6)

In [40]:
# this value came from training method
b_FFN_Expand_1 = np.random.rand(1, 24)
b_FFN_Shrink_2 = np.random.rand(1, 6)

#### *add by using broadCasting*
 * $dataPipes_1 \in R^{(n_{model} \times 5)}$ : result from Plus & Norm process have same dimensions of X 
 * $W_1 \in R^{(n_{model} \times 4n_{model})}$ : layer แรกเป็นเลเยอร์ขยายผลลัพธ์ที่ได้จากเลเยอร์นี้มักจะมีขนาดมิติเป็น 4 เท่าของค่า $n_{model}$

In [41]:
h1 = np.dot(dataPipes1, W_FFN_Expand_1) + b_FFN_Expand_1
h1.shape

(5, 24)

### pass to Relu function

In [42]:
h_prime = np.maximum(0, h1) # if child of h1 less than 0 return 0 instead of minus value inside h1
h_prime

array([[0.11748184, 0.        , 1.59528691, 0.56583539, 1.58861743,
        0.11485782, 1.55723791, 2.06526634, 0.95313585, 1.61946016,
        0.86819807, 1.65793455, 0.        , 0.        , 0.        ,
        1.18226119, 0.        , 1.24669906, 0.17003229, 1.73542133,
        0.        , 0.18737507, 0.30145437, 0.        ],
       [0.        , 0.46880458, 2.60925257, 0.58529624, 0.43549717,
        0.95390211, 0.48503557, 0.36447352, 0.4001374 , 0.40079568,
        0.01705946, 1.02196768, 0.31484362, 0.3047735 , 1.10236711,
        0.        , 0.71950487, 1.33440019, 0.83644753, 1.07725904,
        0.40020528, 0.01599926, 0.        , 0.18928559],
       [0.        , 1.16290576, 1.34178156, 0.41652508, 0.32519056,
        1.14500679, 0.        , 0.19290724, 0.91341151, 0.88804637,
        0.52369191, 0.        , 0.        , 0.64057347, 1.67310265,
        0.        , 2.13351054, 0.88868178, 0.        , 0.21920809,
        0.91735337, 0.71239753, 1.06667884, 0.64870223],
       [0.   

## 2st linear (Shrink)

then dot with shrink weight and shrink bias again

In [43]:
output = np.dot(h_prime, W_FFN_Shrink_1) + b_FFN_Shrink_2
output.shape

(5, 6)

In [44]:
print(output)

[[11.9895897  10.45334526  8.52934202  8.63928045  9.12931467  5.64879593]
 [ 9.16061676  8.21162605  6.94881061  7.10285008  6.15550781  4.88585223]
 [ 8.2997091   7.45603733  7.43069116  8.23319985  7.1303459   6.68903971]
 [ 8.26942016  7.22795236  7.02866046  8.41532928  7.08796718  6.47574114]
 [10.18103016  9.29441969  7.95714925  8.40611695  8.16545352  5.50212623]]


## conclude

In [45]:
def ffn(X, W1, b1, W2, b2):
    h = np.dot(X, W1) + b1
    h = np.maximum(0, h)
    h = np.dot(h, W2) + b2
    return h

In [46]:
dataPipes1 = ffn(dataPipes1, W_FFN_Expand_1, b_FFN_Expand_1, W_FFN_Shrink_1, b_FFN_Shrink_2)
dataPipes1.shape

(5, 6)

# Add and Norm once again at the end of transformer block

In [47]:
addResult = dataPipes1 + X
norm2 = layer_norm(addResult)
norm2.shape

mean =  [[9.39827801]
 [7.24421059]
 [7.70650384]
 [7.7508451 ]
 [8.41771597]]
std =  [[2.18841262]
 [1.40665544]
 [0.66261672]
 [0.71653905]
 [1.69219142]]


(5, 6)

In [48]:
dataPipes1 = norm2
dataPipes1

array([[ 1.64105709,  0.4821151 ,  0.05988996, -0.34682546, -0.12290334,
        -1.71333337],
       [ 1.36238396,  0.68774111,  0.50090413, -0.10049399, -0.77396495,
        -1.67657026],
       [ 0.89524509,  1.13117031, -0.41624706,  0.79487161, -0.86951786,
        -1.53552208],
       [ 0.72372098,  0.6658487 ,  0.38771785,  0.92735107, -0.92510937,
        -1.77952922],
       [ 1.63297871,  0.51808749, -0.2721716 , -0.00685443, -0.14907433,
        -1.72296584]])

# Let's move to final step before we get the next token

after the data pipes pass through transformer block n block (currently n = 1 lol) we got

In [49]:
print(dataPipes1)

[[ 1.64105709  0.4821151   0.05988996 -0.34682546 -0.12290334 -1.71333337]
 [ 1.36238396  0.68774111  0.50090413 -0.10049399 -0.77396495 -1.67657026]
 [ 0.89524509  1.13117031 -0.41624706  0.79487161 -0.86951786 -1.53552208]
 [ 0.72372098  0.6658487   0.38771785  0.92735107 -0.92510937 -1.77952922]
 [ 1.63297871  0.51808749 -0.2721716  -0.00685443 -0.14907433 -1.72296584]]


In [50]:
dataPipes1.shape

(5, 6)

ในการทำนาย Token ถัดไป 

เราจะใช้ hidden state ของตำแหน่งสุดท้ายเท่านั้น

In [51]:
last_hidden = dataPipes1[-1]
print(last_hidden)
print('shape of lass hidden state : ', last_hidden.shape)

[ 1.63297871  0.51808749 -0.2721716  -0.00685443 -0.14907433 -1.72296584]
shape of lass hidden state :  (6,)


ในเชิงความหมายในขั้นตอนนี้

เวคเตอร์นี้คือ "โมเดลเข้าใจประโยคทั้งหมดแล้ว และพร้อมจะเดาคำถัดไป"

Shape : 
$$last\ hidden \in R^{d_{model}}$$ หรือ $$R^{(d_{model}, )}$$

เป้าหมายขั้นสุดท้าย

สมมุติให้ vocab ทั้งหมดมี 20 ตัว
จะได้ output เป็น

$$P(token_1), P(token_2), ..., P(token_{19})$$

ดังนั้นเราต้องแปลง: array จากขนาด $6 \rightarrow 20$

ซึ่ง Transformer block ทำไม่ได้
เป็นหน้าที่ของ LM Head (Output Linear Layer)

## Last Linear layer (LM Head)

Mathemetic definition

$$Logits = h_{last} \cdot W_{vocab} + b_{vocab}$$

Where 
* $h_{last} \in R^{(d_{model}, )}$
* $W_{vocab} \in R^{(d_{model} \times vocab\_size)}$
* $b_{vocab} \in R^{(vocab\_size,)}$ 
* $logits \in R^{(vocab\_size,)}$ 

In [52]:
print("d_model =", d_model)

d_model = 6


In [53]:
vocab_size = 20

W_vocab = np.random.randn(d_model, vocab_size)
b_vocab = np.random.randn(vocab_size)

In [54]:
print(W_vocab.shape)

(6, 20)


In [55]:
logits = np.dot(last_hidden, W_vocab) + b_vocab

In [56]:
print(logits)

[ 1.01721882  1.11217146  1.5821779  -1.83662536 -6.26772262  0.75220777
 -2.06002041 -0.54294198 -1.98382689 -1.82998583 -1.81641587  5.02411645
 -1.2252215   2.47970717 -2.46600529  3.8879696   0.60542558  2.02458777
 -2.21400747 -1.21038219]


if the logit result is   
[ 3.05492994  3.04876717  0.10503566 -0.08815435  2.14139093 -2.3520933
 -3.40995254  4.67951357 -3.81675782 -0.67168809  1.88311049  0.35795136
 -2.11323825 -1.29417076  1.21322951 -0.75312505 -0.32719262 -1.49954131
 -0.33330187 -0.06028637]

ความหมายคือ:
* token #8 (logit = 4.67) มีโอกาสมากกว่า token #1 (logit = 3.05)
* ค่ายังไม่ใช่ probability
* เป็นแค่คะแนนความเหมาะสม

### Softmax -> probability

Softmax ทำแค่ "normalize"

In [57]:
probs = softmax(logits)
probs.shape

(20,)

In [58]:
probs

array([1.16638112e-02, 1.28256058e-02, 2.05210269e-02, 6.72096602e-04,
       7.99890633e-06, 8.94844643e-03, 5.37542075e-04, 2.45059338e-03,
       5.80100029e-04, 6.76573857e-04, 6.85817513e-04, 6.41230265e-01,
       1.23868542e-03, 5.03490290e-02, 3.58175164e-04, 2.05869355e-01,
       7.72682277e-03, 3.19400254e-02, 4.60825744e-04, 1.25720371e-03])

### ความหมายเชิงผลลัพธ์

In [59]:
for i, p in enumerate(probs):
    print(f"token_{i}: {p:.4f}")

token_0: 0.0117
token_1: 0.0128
token_2: 0.0205
token_3: 0.0007
token_4: 0.0000
token_5: 0.0089
token_6: 0.0005
token_7: 0.0025
token_8: 0.0006
token_9: 0.0007
token_10: 0.0007
token_11: 0.6412
token_12: 0.0012
token_13: 0.0503
token_14: 0.0004
token_15: 0.2059
token_16: 0.0077
token_17: 0.0319
token_18: 0.0005
token_19: 0.0013


In [60]:
next_token_id = np.argmax(probs)
next_token_id

np.int64(11)

In [61]:
next_token_id = np.random.choice(
    range(vocab_size),
    p=probs
)

In [62]:
next_token_id

np.int64(15)